In [8]:
import os
os.chdir('/tf/shared')
from tqdm import tqdm

In [2]:
from DataPrep import getExposedItemsTrainSet
recItems, purLabels = getExposedItemsTrainSet()

Number of Multiprocessing threads: 31


/tmp/ipykernel_1025488/3715054922.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  recItems, purLabels = getExposedItemsTrainSet()


In [16]:
# get click history
import pandas as pd
rawTrainSet = pd.read_csv('/tf/shared/trainset.csv',' ')
clickHistRaw = rawTrainSet.user_click_history
clickHist = []
for text in tqdm(clickHistRaw):
    clickSeries = text.split(',')
    clickItems  = [int(x.partition(':')[0]) for x in clickSeries]
    clickHist.append(clickItems)

100% 260087/260087 [00:06<00:00, 42637.24it/s]


In [20]:
##### calculate probability that an item was clicked given it was recommeneded
countRec = 0
countClickedAndRec = 0

for index, sample in enumerate(recItems):
# loop thru samples
    clickedItems = clickHist[index]
    for item in sample:
        countRec = countRec + 1
        if item in clickedItems:
            countClickedAndRec = countClickedAndRec + 1

print('probability of clicked given recommended: ' + str(countClickedAndRec/countRec))


probability of clicked given recommended: 0.2715463159122396


In [23]:
##### calculate general probability of purchase ##########
countPur = 0
countRec = 0
for index, sample in enumerate(recItems):
# loop thru samples
    clickedItems = clickHist[index]
    labels = purLabels[index]
    countRec = countRec + 9
    for j in range(9):
        if j>2 and labels[0]*labels[1]*labels[2]==0:
            break
        if j>5 and labels[3]*labels[4]*labels[5]==0:
            break
        if labels[j]==1:
            countPur = countPur + 1

print('general probability of purchase: ' + str(countPur/countRec))


general probability of purchase: 0.5937675555572643


In [26]:
##### calculate probability that a recommended item was purchased given it was clicked
countClicked = 0
countClickedAndPur = 0

for index, sample in enumerate(recItems):
# loop thru samples
    clickedItems = clickHist[index]
    labels = purLabels[index]
    for j in range(9):
        if j>2 and labels[0]*labels[1]*labels[2]==0:
            break
        if j>5 and labels[3]*labels[4]*labels[5]==0:
            break
        item = sample[j]
        if item in clickedItems:
            countClicked = countClicked + 1
            if labels[j]==1:
                countClickedAndPur = countClickedAndPur + 1

print('count Clicks: ' + str(countClicked))
print('count Clicks and Purchases: ' + str(countClickedAndPur))
print('probability of purchase given clicked: ' + str(countClickedAndPur/countClicked))

count Clicks: 596721
count Clicks and Purchases: 548193
probability of purchase given clicked: 0.9186755619460351
